conda create -n llm-dev python=3.9 ipykernel --yes


pip install -r requirements.txt --no-cache-dir


requirements:


    faiss-cpu==1.7.4

    chromadb

    --transformers

In [ ]:
import chromadb
import pandas as pd

In [ ]:
chroma_client = chromadb.Client()

In [ ]:
# from chromadb.config import Settings
# chroma_client = chromadb.Client(
#     Settings(
#         chroma_db_impl="duckdb+parquet",
#         persist_directory='my_db_folder',
#     )
# )

In [ ]:
collection_name = "my_news"

In [ ]:
from time import time

In [ ]:
time?

In [ ]:
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
    chroma_client.delete_collection(name=collection_name)
else:
    print(f"Creating collection: '{collection_name}'")
    collection = chroma_client.create_collection(name=collection_name)


In [ ]:
pdf = pd.read_csv("labelled_newscatcher_coloured.csv", index_col=0)

In [ ]:
pdf_subset = pdf.head(1000)

In [ ]:
pdf_subset

In [ ]:
import time
# inicio = time.time()
#------------------
collection.add(
    documents=pdf_subset["title"][:100].to_list(),
    metadatas=[{"topic": topic} for topic in pdf_subset["topic"][:100].tolist()],
    ids=[f"id{x}" for x in range(100)],
)
# fin = time.time()
# print(fin-inicio)

CRUD

In [ ]:
import json
inicio = time.time()

results = collection.query(
    query_texts=["space"],
    # query_texts=["espacio"],
    n_results=10
)

fin = time.time()

print(json.dumps(results, indent=4))

In [ ]:
fin-inicio

In [ ]:
import json

results = collection.query(
    query_texts=["bombs"],
    # query_texts=["espacio"],
    n_results=3
)

print(json.dumps(results, indent=4))

In [ ]:
collection.query(
    query_texts=["space"],
    where={"topic": "SCIENCE"},
    n_results=10,
)

In [ ]:
collection.delete(
    ids=["id0"],
)

In [ ]:
collection.get(
    ids=["id0"],
)

In [ ]:
collection.get(
    ids=["id2"],
)

In [ ]:
collection.update(
    ids=["id2"],
    metadatas={"topic": "TECHNOLOGY"}
)

In [ ]:
collection.get(
    ids=["id2"],
)

pip3 install torch torchvision torchaudio

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    # cache_dir='cache'
)

lm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # cache_dir='cache',
)

pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device_map="auto",
)

In [ ]:
results

In [ ]:
import json

results = collection.query(
    query_texts=["space"],
    # query_texts=["espacio"],
    n_results=10
)

print(json.dumps(results, indent=4))

In [ ]:
question = "What's the latest news on space deveolpment?"

In [ ]:
context = " ".join([f"\n#{str(i)}" for i in results["documents"][0]])
print(context)

In [ ]:
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
print(prompt_template)

In [ ]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

OPEN AI
https://docs.trychroma.com/embeddings